In [1]:
# TODO: convert to .py

from PIL import Image
import numpy as np

In [3]:
im_path = 'test_data/test.png'
mask_path = 'make_bad_mask/test_mask_bad.png'

im = Image.open(im_path)
mask = Image.open(mask_path)

In [7]:
print(im.size)
print(mask.size)

(6359, 1943)
(6359, 1943)
